## Combining New Data

The primary goal of this notebook is to seamlessly integrate newly scraped data with our existing datasets to ensure comprehensive predictions for upcoming matches. This process closely mirrors the steps outlined in previous notebooks, but with the added complexity of incorporating real-time data to enrich our analyses and predictions.

### Purpose of Combining New Data

As we continue to refine our predictive models, it is essential to ensure that all historical data remains consistent and accurate, especially when new information about upcoming matches becomes available. By incorporating this new data, we aim to:

1. **Enhance Predictive Accuracy**: Updating our datasets with the latest odds and match details allows for a more nuanced understanding of the current betting landscape. This, in turn, leads to better predictions regarding match outcomes.

2. **Maintain Consistency**: It is crucial that the historical data for past matches is consistent with the newly added data. This includes ensuring that feature engineering processes applied to historical data are also applied to the new data, allowing for meaningful comparisons and analyses.

3. **Update Feature Engineering**: We will rerun our feature engineering process to create new variables that reflect the most recent match statistics, player forms, and betting odds. This will involve recalculating metrics such as:
   - Goals scored and conceded
   - Average goals per match
   - Expected goals based on recent performances
   - Odds adjustments reflecting the current market

In [28]:
import pandas as pd
import numpy as np
import warnings
from Dataset_functions import *
import re 
# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [30]:
teams = ['Arsenal', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton',
       'Southampton', 'Watford', 'West Brom', 'Man United', 'Newcastle',
       'Swansea', 'Stoke', 'Burnley', 'Leicester', 'Bournemouth',
       'Liverpool', 'Huddersfield', 'Tottenham', 'Man City', 'West Ham',
       'Hamburg', 'Reading', 'Bochum', 'Ipswich', 'Millwall', 'Preston',
       'Sheffield United', 'Wigan', 'Bristol City', 'Brentford',
       'Greuther Furth', 'Regensburg', 'Birmingham', 'Magdeburg', 'Leeds',
       'Darmstadt', 'Heidenheim', 'Union Berlin', 'Hull', 'Dresden',
       "Nott'm Forest", 'Middlesbrough', 'St Pauli', 'Paderborn',
       'Ingolstadt', 'Aston Villa', 'Derby', 'Bolton', 'Norwich', 'QPR',
       'Sheffield Weds', 'Duisburg', 'Bielefeld', 'Blackburn',
       'Rotherham', 'Fulham', 'Wolves', 'Erzgebirge Aue', 'Holstein Kiel',
       'Sandhausen', 'FC Koln', 'Girona', 'Cadiz', 'Albacete', 'Betis',
       'Cardiff', 'Alcorcon', 'Cordoba', 'Elche', 'Lugo', 'Lazio',
       'Chievo', 'Barcelona', 'Celta', 'Villarreal', 'Mallorca',
       'Las Palmas', 'Oviedo', 'Zaragoza', 'Parma', 'Sassuolo', 'Torino',
       'Bologna', 'Empoli', 'Vallecano', 'Real Madrid', 'Eibar',
       'Atalanta', 'Gimnastic', 'Valencia', 'Ath Bilbao', 'Leganes',
       'Getafe', 'Bayern Munich', 'Extremadura UD', 'Malaga', 'Wolfsburg',
       'Werder Bremen', "M'gladbach", 'Hertha', 'Freiburg', 'Alaves',
       'Valladolid', 'Ath Madrid', 'Reus Deportiu', 'Fortuna Dusseldorf',
       'Juventus', 'Napoli', 'Numancia', 'Sevilla', 'Mainz', 'Espanol',
       'Dortmund', 'Almeria', 'Osasuna', 'Granada', 'Cagliari',
       'Frosinone', 'Genoa', 'Fiorentina', 'Spal', 'Udinese', 'Sp Gijon',
       'Inter', 'Rayo Majadahonda', 'Levante', 'Roma', 'Milan',
       'Hannover', 'Tenerife', 'Leverkusen', 'Hoffenheim',
       'Ein Frankfurt', 'Augsburg', 'Nurnberg', 'Stuttgart', 'Sampdoria',
       'RB Leipzig', 'Schalke 04', 'La Coruna', 'Huesca', 'Sociedad',
       'Osnabruck', 'Wehen', 'Luton', 'Karlsruhe', 'Barnsley', 'Charlton',
       'Santander', 'Pisa', 'Mirandes', 'Crotone', 'Salernitana',
       'Cittadella', 'Venezia', 'Virtus Entella', 'Ascoli',
       'Ponferradina', 'Perugia', 'Verona', 'Pordenone', 'Livorno',
       'Cremonese', 'Spezia', 'Benevento', 'Cosenza', 'Fuenlabrada',
       'Trapani', 'Lecce', 'Juve Stabia', 'Pescara', 'Brescia', 'Wycombe',
       'Coventry', 'Wurzburger Kickers', 'Castellon', 'Cartagena',
       'Monza', 'Logrones', 'Braunschweig', 'Reggiana', 'Reggina',
       'Vicenza', 'Sabadell', 'Hansa Rostock', 'Peterboro', 'Sociedad B',
       'Blackpool', 'Ternana', 'Ibiza', 'Amorebieta', 'Burgos', 'Como',
       'Alessandria', 'Kaiserslautern', 'Sunderland', 'Palermo', 'Modena',
       'Villarreal B', 'Bari', 'Sudtirol', 'Andorra', 'Plymouth',
       'Elversberg', 'FeralpiSalo', 'Catanzaro', 'Ferrol', 'Eldense',
       'Lecco', 'Ulm', 'Oxford', 'Preußen Münster', 'Portsmouth']

### Updating Historical Head-to-Head (H2H) Data

To ensure that our predictive models are based on the most accurate and comprehensive datasets, we will re-run the steps for each league to gain an overall insight into the data and update the historical head-to-head (H2H) records. This process is crucial for understanding team dynamics and rivalries, which can significantly impact match outcomes.

#### Purpose of Updating H2H Data

The primary objectives of updating the historical H2H data include:

1. **Enhancing Model Accuracy**: Historical performance between teams can reveal trends and patterns that are not captured in aggregate statistics. By refreshing the H2H data, we gain valuable insights into how teams perform against each other, which can be a strong predictor of future results.

2. **Incorporating Recent Matches**: By analyzing the latest matchups, we can adjust our predictions to reflect recent performances, injuries, or tactical changes that may influence upcoming games. This is particularly important in dynamic sports environments where team forms can fluctuate rapidly.

3. **Identifying Key Factors**: Updating the H2H data allows us to identify specific factors that may influence outcomes, such as:
   - Historical win/loss records
   - Goals scored and conceded in past matchups
   - Home versus away performance in H2H scenarios
   - Trends in match results over specific periods

In [32]:
premier_league_season17_19 = prepare_dataframe('17_18_eng.csv', '2017-2018')
premier_league_season18_19 = prepare_dataframe('18_19_eng.csv', '2018-2019')
premier_league_season19_20 = prepare_dataframe('19_20_eng.csv', '2019-2020')
premier_league_season20_21 = prepare_dataframe('20_21_eng.csv', '2020-2021')
premier_league_season21_22 = prepare_dataframe('21_22_eng.csv', '2021-2022')
premier_league_season22_23 = prepare_dataframe('22_23_eng.csv', '2022-2023')
premier_league_season23_24 = prepare_dataframe('23_24_eng.csv', '2023-2024')
premier_league_season24_25 = prepare_dataframe('24_25_eng_v1.csv', '2024-2025')

premier_league_data = pd.concat([premier_league_season17_19, premier_league_season18_19, premier_league_season19_20, premier_league_season20_21, premier_league_season21_22, premier_league_season22_23, premier_league_season23_24,premier_league_season24_25])
premier_league_data.fillna(0, inplace = True)
# premier_league_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']] = season_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']]
premier_league_data.to_csv('premier_league_data_v1.csv', index = False)

In [33]:
spanish1_season18_19 = prepare_dataframe_spanish1('SP1_18_19.csv', '2018-2019')
spanish1_season19_20 = prepare_dataframe_spanish1('SP1_19_20.csv', '2019-2020')
spanish1_season20_21 = prepare_dataframe_spanish1('SP1_20_21.csv', '2020-2021')
spanish1_season21_22 = prepare_dataframe_spanish1('SP1_21_22.csv', '2021-2022')
spanish1_season22_23 = prepare_dataframe_spanish1('SP1_22_23.csv', '2022-2023')
spanish1_season23_24 = prepare_dataframe_spanish1('SP1_23_24.csv', '2023-2024')
spanish1_season24_25 = prepare_dataframe_spanish1('SP_24_25_v1.csv', '2024-2025')

In [34]:
spanish1_data = pd.concat([spanish1_season18_19, spanish1_season19_20, spanish1_season20_21, spanish1_season21_22, spanish1_season22_23, spanish1_season23_24,spanish1_season24_25])
spanish1_data.fillna(0, inplace = True)
spanish1_data.to_csv('spanish1_data_v1.csv', index = False)

In [35]:
# spanish1_season17_18 = prepare_dataframe_spanish1('SP1_17_18.csv', '2017-2018')
italian1_season24_25 = prepare_dataframe_italian1('I1_24_25_v1.csv', '2024-2025')
italian1_season18_19 = prepare_dataframe_italian1('I1_18_19.csv', '2018-2019')
italian1_season19_20 = prepare_dataframe_italian1('I1_19_20.csv', '2019-2020')
italian1_season20_21 = prepare_dataframe_italian1('I1_20_21.csv', '2020-2021')
italian1_season21_22 = prepare_dataframe_italian1('I1_21_22.csv', '2021-2022')
italian1_season22_23 = prepare_dataframe_italian1('I1_22_23.csv', '2022-2023')
italian1_season23_24 = prepare_dataframe_italian1('I1_23_24.csv', '2023-2024')


In [36]:
italian1_data = pd.concat([italian1_season18_19, italian1_season19_20, italian1_season20_21, italian1_season21_22, italian1_season22_23, italian1_season23_24,italian1_season24_25])
italian1_data.fillna(0, inplace = True)
italian1_data.to_csv('italian1_data_v1.csv', index = False)

In [37]:
german1_season18_19 = prepare_dataframe_german1('D1_18_19.csv', '2018-2019')
german1_season19_20 = prepare_dataframe_german1('D1_19_20.csv', '2019-2020')
german1_season20_21 = prepare_dataframe_german1('D1_20_21.csv', '2020-2021')
german1_season21_22 = prepare_dataframe_german1('D1_21_22.csv', '2021-2022')
german1_season22_23 = prepare_dataframe_german1('D1_22_23.csv', '2022-2023')
german1_season23_24 = prepare_dataframe_german1('D1_23_24.csv', '2023-2024')
german1_season24_25 = prepare_dataframe_german1('D1_24_25_v1.csv', '2024-2025')

In [38]:
german1_data = pd.concat([german1_season18_19, german1_season19_20, german1_season20_21, german1_season21_22, german1_season22_23, german1_season23_24,german1_season24_25])
german1_data.fillna(0, inplace = True)
german1_data.to_csv('german1_data_v1.csv', index = False)

In [39]:
season_data0 = pd.read_csv('24_25_eng_v1.csv')
season_data00 = pd.read_csv('23_24_eng.csv')
season_data2 = pd.read_csv('22_23_eng.csv')
season_data3 = pd.read_csv('21_22_eng.csv')
season_data4 = pd.read_csv('20_21_eng.csv')
season_data5 = pd.read_csv('19_20_eng.csv')
season_data6 = pd.read_csv('18_19_eng.csv')
season_data7 = pd.read_csv('17_18_eng.csv')

championship_data0 = pd.read_csv('championship_24_25.csv')
championship_data1 = pd.read_csv('championship_23_24.csv')
championship_data2 = pd.read_csv('championship_22_23.csv')
championship_data3 = pd.read_csv('championship_21_22.csv')
championship_data4 = pd.read_csv('championship_20_21.csv')
championship_data5 = pd.read_csv('championship_19_20.csv')
championship_data6 = pd.read_csv('championship_18_19.csv')

spanish1_data0 = pd.read_csv('SP_24_25_v1.csv')
spanish1_data1 = pd.read_csv('SP1_23_24.csv')
spanish1_data2 = pd.read_csv('SP1_22_23.csv')
spanish1_data3 = pd.read_csv('SP1_21_22.csv')
spanish1_data4 = pd.read_csv('SP1_20_21.csv')
spanish1_data5 = pd.read_csv('SP1_19_20.csv')
spanish1_data6 = pd.read_csv('SP1_18_19.csv')

# spanish2_data0 = pd.read_csv('SP2_24_25.csv')
spanish2_data1 = pd.read_csv('SP2_23_24.csv')
spanish2_data2 = pd.read_csv('SP2_22_23.csv')
spanish2_data3 = pd.read_csv('SP2_21_22.csv')
spanish2_data4 = pd.read_csv('SP2_20_21.csv')
spanish2_data5 = pd.read_csv('SP2_19_20.csv')
spanish2_data6 = pd.read_csv('SP2_18_19.csv')

italian1_data0 = pd.read_csv('I1_24_25_v1.csv')
italian1_data1 = pd.read_csv('I1_23_24.csv')
italian1_data2 = pd.read_csv('I1_22_23.csv')
italian1_data3 = pd.read_csv('I1_21_22.csv')
italian1_data4 = pd.read_csv('I1_20_21.csv')
italian1_data5 = pd.read_csv('I1_19_20.csv')
italian1_data6 = pd.read_csv('I1_18_19.csv')

italian2_data1 = pd.read_csv('I2_23_24.csv')
italian2_data2 = pd.read_csv('I2_22_23.csv')
italian2_data3 = pd.read_csv('I2_21_22.csv')
italian2_data4 = pd.read_csv('I2_20_21.csv')
italian2_data5 = pd.read_csv('I2_19_20.csv')
# italian2_data6 = pd.read_csv('I2_18_19.csv')

german1_data0 = pd.read_csv('D1_24_25_v1.csv')
german1_data1 = pd.read_csv('D1_23_24.csv')
german1_data2 = pd.read_csv('D1_22_23.csv')
german1_data3 = pd.read_csv('D1_21_22.csv')
german1_data4 = pd.read_csv('D1_20_21.csv')
german1_data5 = pd.read_csv('D1_19_20.csv')
german1_data6 = pd.read_csv('D1_18_19.csv')

german2_data0 = pd.read_csv('D2_24_25.csv')
german2_data1 = pd.read_csv('D2_23_24.csv')
german2_data2 = pd.read_csv('D2_22_23.csv')
german2_data3 = pd.read_csv('D2_21_22.csv')
german2_data4 = pd.read_csv('D2_20_21.csv')
german2_data5 = pd.read_csv('D2_19_20.csv')
german2_data6 = pd.read_csv('D2_18_19.csv')

season_data = pd.concat([season_data7, season_data6,season_data5,season_data4,season_data3,season_data2,season_data00,season_data0,
                        championship_data0, championship_data1, championship_data2, championship_data3, championship_data4, championship_data5, championship_data6,
                        spanish1_data0, spanish1_data1,spanish1_data2,spanish1_data3,spanish1_data4,spanish1_data5,spanish1_data6,
                          spanish2_data1,spanish2_data2,spanish2_data3,spanish2_data4,spanish2_data5,spanish2_data6,
                        italian1_data0, italian1_data1,italian1_data2,italian1_data3,italian1_data4,italian1_data5,italian1_data6,
                         italian2_data2,italian2_data3,italian2_data4,italian2_data5,italian2_data1,
                         german1_data0, german1_data1,german1_data2,german1_data3,german1_data4,german1_data5,german1_data6,
                          german2_data0, german2_data1,german2_data2,german2_data3,german2_data4,german2_data5,german2_data6,
                        ])




# Apply the function to the date column
# season_data['Date'] = pd.to_datetime(season_data['Date'],format = '%d/%m/%Y') 
season_data['Date'] = season_data['Date'].str[:10]
season_data['Date'] = season_data['Date'].apply(standardize_date_format)
season_data['Date'] = pd.to_datetime(season_data['Date'],format = '%d/%m/%Y') 
# season_data['Date'] = pd.to_datetime(season_data['Date'], format='%Y-%m-%d', errors='coerce')
# season_data['Date'] = season_data['Date'].fillna(pd.to_datetime(season_data['Date'], format='%d/%m/%Y', errors='coerce'))
season_data.sort_values(['Date'],ascending = True, inplace = True)
season_data.reset_index(drop = True, inplace = True)
season_data =  get_team_h2h(season_data)

In [40]:
season_data.to_csv('h2h_data_v1.csv', index = False)

### Updating Team Elo Ratings

In addition to refreshing the historical head-to-head (H2H) data, it is essential to obtain the most recent information regarding team Elo ratings. Elo ratings are a widely recognized measure of a team's relative skill level and can fluctuate significantly based on match results. By incorporating up-to-date Elo ratings, we can enhance our models' predictive power and provide a more accurate reflection of each team's current form.

In [41]:
team_dfs = {}
for i in teams:
    try:
        team_dfs[i] =  pd.read_csv(f'{i}.csv')
    except:
        pass
h2h_data = pd.read_csv('h2h_data_v1.csv')
h2h_data['Date'] = pd.to_datetime(h2h_data['Date'])

for team_df in team_dfs.values():
    team_df['From'] = pd.to_datetime(team_df['From'])
    team_df['To'] = pd.to_datetime(team_df['To'])

h2h_data['home_elo'] = None
h2h_data['away_elo'] = None
for index, row in h2h_data.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    match_date = row['Date']

    h2h_data.at[index, 'home_elo'] = find_elo_from_team_df(home_team, match_date, team_dfs)
    h2h_data.at[index, 'away_elo'] = find_elo_from_team_df(away_team, match_date, team_dfs)

h2h_data['home_elo'].fillna(1400, inplace = True)
h2h_data['away_elo'].fillna(1400, inplace = True)

h2h_data[['Avg>2.5', 'Avg<2.5']].to_csv('over_under_2425.csv',index = False)

h2h_data['month'] = h2h_data['Date'].dt.month.astype(str)
h2h_data['year'] = h2h_data['Date'].dt.year.astype(str)


h2h_data['over_2.5_goals'] = (h2h_data['FTHG'] + h2h_data['FTAG']) > 2.5

h2h_data['home_over_2.5_ratio'] = 0.0
h2h_data['away_over_2.5_ratio'] = 0.0

# Iterate through each row and calculate the over 2.5 ratios for home and away teams
for index, row in h2h_data.iterrows():
    # Home team matches prior to the current match
    home_team_matches = h2h_data[(h2h_data['HomeTeam'] == row['HomeTeam']) & (h2h_data['Date'] < row['Date'])]
    if len(home_team_matches) > 0:
        home_over_2_5_ratio = home_team_matches['over_2.5_goals'].mean()
    else:
        home_over_2_5_ratio = 0  # No prior matches

    # Away team matches prior to the current match
    away_team_matches = h2h_data[(h2h_data['AwayTeam'] == row['AwayTeam']) & (h2h_data['Date'] < row['Date'])]
    if len(away_team_matches) > 0:
        away_over_2_5_ratio = away_team_matches['over_2.5_goals'].mean()
    else:
        away_over_2_5_ratio = 0  # No prior matches

    # Assign the calculated ratios
    h2h_data.at[index, 'home_over_2.5_ratio'] = home_over_2_5_ratio
    h2h_data.at[index, 'away_over_2.5_ratio'] = away_over_2_5_ratio

In [42]:
h2h_data.to_csv('h2h_data_v1.csv', index = False)

### Achieving Updated Data Format for Future Matches

After completing the steps to update both the historical head-to-head (H2H) data and the team Elo ratings, we have successfully transformed our datasets to reflect the most current information available. This ensures that our data is now formatted in the same way as after the initial iteration of data combining, allowing us to maintain consistency in our analysis.

#### Importance of Updated Data Format

The updated data format is critical for several reasons:

1. **Consistency for Modeling**: By ensuring that the structure and format of our datasets are consistent with our previous iterations, we facilitate a seamless transition to the modeling phase. This consistency helps to avoid issues related to mismatched data types or variable names, which can complicate the modeling process.

2. **Enhanced Predictive Accuracy**: The incorporation of updated H2H data and Elo ratings ensures that our model inputs are based on the latest performance metrics. This increases the likelihood of producing accurate predictions, as the model can leverage the most relevant information when assessing upcoming fixtures.

3. **Comprehensive Analysis**: With the updated datasets, we can conduct a more thorough analysis of the factors influencing match outcomes. This includes examining how recent performances, historical rivalries, and current team strengths may impact the results.


### Summary

With the completion of the updates to both the H2H data and team Elo ratings, we now possess a fully updated dataset that mirrors the initial format established in our previous iterations. This data is crucial for modeling and predicting match outcomes, providing a solid foundation for our analytical efforts. By leveraging the most current information, we enhance our ability to generate accurate predictions and make informed betting decisions in the competitive landscape of football matches.